In [1]:
# Импорт необходимых библиотек
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pickle   
import joblib

In [9]:
# Определяем устройство для вычислений (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Будет использоваться устройство:", device)
# количество отображаемых примеров
number_of_samples = 10

Будет использоваться устройство: cuda


In [3]:
MODEL_DIR = '../models/final_model'

if not os.path.isdir(MODEL_DIR):
    raise FileNotFoundError(f"Каталог с моделью не найден: {MODEL_DIR}")

# 1) пытаемся загрузить как HF-трансформер
model = None
tokenizer = None
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
    print("✅ Успешно загружены токенизатор и модель Transformers из локальной папки.")
except Exception as hf_err:
    print("⚠️ Не удалось загрузить как Transformers-модель:", hf_err)

# 2) если не трансформер — ищем файлы с другими расширениями
if model is None:
    # список файлов в папке
    files = os.listdir(MODEL_DIR)
    # ищем PyTorch state-dict (.pt/.pth)
    pt_files = [f for f in files if f.endswith(('.pt','.pth'))]
    # ищем sklearn pickle/joblib (.pkl/.joblib/.sav)
    pkl_files = [f for f in files if f.endswith(('.pkl','.joblib','.sav'))]
    # ищем Keras H5 (.h5)
    h5_files  = [f for f in files if f.endswith('.h5')]
    
    if pt_files:
        pt_path = os.path.join(MODEL_DIR, pt_files[0])
        try:
            model = torch.load(pt_path, map_location='cpu')
            print(f"✅ Загружен PyTorch-модель из файла {pt_files[0]} на CPU.")
        except Exception as e:
            print(f"❌ Ошибка при загрузке PyTorch-модели {pt_files[0]}:", e)
    elif pkl_files:
        pkl_path = os.path.join(MODEL_DIR, pkl_files[0])
        try:
            model = pickle.load(open(pkl_path, 'rb'))
            print(f"✅ Загружена модель sklearn (pickle) из {pkl_files[0]}.")
        except Exception:
            # пробуем joblib.load
            try:
                model = joblib.load(pkl_path)
                print(f"✅ Загружена модель sklearn (joblib) из {pkl_files[0]}.")
            except Exception as e2:
                print(f"❌ Ошибка при загрузке sklearn-модели из {pkl_files[0]}:", e2)
    elif h5_files:
        from tensorflow import keras
        h5_path = os.path.join(MODEL_DIR, h5_files[0])
        try:
            model = keras.models.load_model(h5_path)
            print(f"✅ Загружена Keras-модель из файла {h5_files[0]}.")
        except Exception as e:
            print(f"❌ Ошибка при загрузке Keras-модели {h5_files[0]}:", e)
    else:
        print("❌ В папке нет файлов с известными расширениями .pt/.pth/.pkl/.joblib/.h5.")

# 3) если модель загрузилась — размещаем на нужном устройстве
if isinstance(model, AutoModelForSequenceClassification):
    # определяем устройство
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        try:
            torch.cuda.empty_cache()
            model.to(device)
            print(f"✅ Трансформер перемещён на {device}.")
        except RuntimeError as oom:
            if 'out of memory' in str(oom):
                print("⚠️ CUDA OOM — оставляем модель на CPU.")
                device = torch.device('cpu')
                model.to(device)
            else:
                raise
    else:
        print("ℹ️ GPU недоступен, используем CPU.")
    model.eval()
else:
    # для sklearn/keras/PyTorch state-dict устройство управляется в процессе инференса
    device = torch.device('cpu')
    print("ℹ️ Пользуемся sklearn/keras/PyTorch state-dict, инференс будет на CPU.")

# Финальное состояние
print("\n== Итоги загрузки модели ==")
print("Model object type:", type(model))
print("Device:", device)


✅ Успешно загружены токенизатор и модель Transformers из локальной папки.
ℹ️ Пользуемся sklearn/keras/PyTorch state-dict, инференс будет на CPU.

== Итоги загрузки модели ==
Model object type: <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>
Device: cpu


In [4]:
# Пытаемся загрузить тестовые данные из data/processed
test_df = None
# Проверяем доступность файлов с данными
if  os.path.exists('../data/processed/final.xlsx'):
    # берем данные из обработанного датасета
    full_df = pd.read_excel('../data/processed/final.xlsx')
    # Удаляем лишний индекс-столбец, если он есть
    if 'Unnamed: 0' in full_df.columns:
        full_df = full_df.drop('Unnamed: 0', axis=1)
    test_df = full_df

if test_df is not None and not test_df.empty:
    # Выбираем одну случайную строку без фиксации random_state
    test_df = test_df.sample(n=number_of_samples).reset_index(drop=True)
    print("Выбрана случайная строка из тестовых данных:")
    display(test_df.head())
else:
    print("Тестовые данные не найдены или они пустые, будет сгенерирован пример вручную.")
    # Генерируем пример вручную
    test_df = pd.DataFrame({
        'full_text': [
            "Президент провёл заседание по вопросам экономической политики."
        ]
    })
    display(test_df.head())


Выбрана случайная строка из тестовых данных:


,full_text,Спорт,Личная жизнь,Юмор,Соцсети,Политика,Реклама,Нет категории
0,еще больше стихов в тг: kkesler 🫀 здесь все п...,0.0,0.5,0.0,0.5,0.0,0.0,0.0
1,"потрясающее сырное печенье, очень рекомендую к...",0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,вы когда-нибудь покупали курсы? курс от соболе...,0.0,0.3,0.0,0.0,0.0,0.7,0.0
3,"еще один герой💔 школьник ислам, который подраб...",0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,"на данный момент, запасов крови и плазмы доста...",0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
# Подготовка списка текстовых примеров для предсказания
text_examples = []
true_labels = None  # сюда сохраним истинные метки, если они известны (для демонстрации)

if test_df is not None:
    # Определяем имя колонки с текстом (предположим, 'full_text')
    text_col = 'full_text' if 'full_text' in test_df.columns else test_df.columns[0]
    # Получаем тексты (возьмём до 5 примеров для наглядности)
    text_examples = test_df[text_col].astype(str).tolist()[:number_of_samples]
    # Если в данных присутствуют столбцы с метками классов (например, многоклассовые бинарные индикаторы)
else:
    # Генерируем пример текста вручную
    text_examples = [
        "Президент провёл заседание по вопросам экономической политики.",  # ожидание категории "Политика"
        "Этот фильм был настолько смешным, что я смеялся весь вечер."      # ожидание категории "Юмор"
    ]
    print("Примеры текстов для предсказания (сгенерированы вручную).")
    
# Выводим сами тексты для проверки
for i, text in enumerate(text_examples, 1):
    print(f"{i}) {text[:60]}{'...' if len(text) > 60 else ''}")

1) еще больше стихов в тг: kkesler 🫀  здесь все про саморазвити...
2) потрясающее сырное печенье, очень рекомендую к приготовлению...
3) вы когда-нибудь покупали курсы? курс от соболева l киев вы д...
4) еще один герой💔 школьник ислам, который подрабатывал в гарде...
5) на данный момент, запасов крови и плазмы достаточно для оказ...
6) вот уже два года я не комментировала никак ситуацию с допинг...
7) hala madrid!!!!! 🤍🤍🤍   https://t.me/tashcin/1734  п александ...
8) @savarastt делал все, чтобы я не падал. но я падал, падал и ...
9) zhenforum https://t.me/zhenforum/6294    вы часто просите на...
10) увидел эту новость:  пресс-конференция сборной нидерландов о...


In [6]:
# Определяем max_length для токенизации
# Если tokenizer.model_max_length задан и разумен — используем его, иначе ставим 512
max_len = getattr(tokenizer, 'model_max_length', None)
if not isinstance(max_len, int) or max_len > 10000:
    max_len = 512
print(f"Будем использовать max_length = {max_len}")

# Токенизация с явным указанием max_length
inputs = tokenizer(
    text_examples,
    padding='max_length',   # паддинг до max_length
    truncation=True,        # обрезаем тексты длиннее max_length
    max_length=max_len,     # максимальная длина
    return_tensors='pt'     # PyTorch-тензоры
)

# Переносим тензоры на устройство
inputs = {k: v.to(device) for k, v in inputs.items()}

print("✅ Токенизация выполнена. Ключи в inputs:", inputs.keys())


Будем использовать max_length = 512
✅ Токенизация выполнена. Ключи в inputs: dict_keys(['input_ids', 'attention_mask'])


In [7]:
# Получаем предсказания модели на токенизированных данных
with torch.no_grad():  # выключаем вычисление градиентов, т.к. сейчас инференс
    outputs = model(**inputs)
    logits = outputs.logits  # логиты (выходы до применения функции активации) размером [batch_size, num_labels]

# Преобразуем логиты в вероятности сигмоидой, т.к. это multi-label классификация
probs_tensor = torch.sigmoid(logits)
# Переносим на CPU и в numpy для дальнейшей обработки
probs = probs_tensor.cpu().numpy()

# Определяем предсказанные метки при пороге 0.5
pred_labels = (probs >= 0.5).astype(int)

print("Предсказания получены. Форма массива вероятностей:", probs.shape)


Предсказания получены. Форма массива вероятностей: (10, 7)


In [8]:
# Определяем список названий классов/тем для вывода (если не определён из данных)
label_cols = ['Спорт', 'Личная жизнь', 'Юмор', 'Соцсети', 'Политика', 'Реклама', 'Нет категории']

# Проходим по каждому примеру и выводим информацию
for i, text in enumerate(text_examples):
    print(f"\nПример {i+1}. Текст: {text}")
    # Истинные категории (если известны)
    if true_labels is not None:
        true_topics = [label_cols[j] for j, val in enumerate(true_labels[i]) if val == 1]
        print("Истинные категории:", ", ".join(true_topics) if true_topics else "нет")
    # Предсказанные категории
    pred_topics = [label_cols[j] for j, val in enumerate(pred_labels[i]) if val == 1]
    print("Предсказанные категории:", ", ".join(pred_topics) if pred_topics else "нет")
    # Вероятности по каждой категории
    probs_str = ", ".join(f"{label_cols[j]}: {p:.2f}" for j, p in enumerate(probs[i]))
    print("Вероятности (по порядку категорий):", probs_str)



Пример 1. Текст: еще больше стихов в тг: kkesler 🫀  здесь все про саморазвитие, поэзию и психологию   подписывайся на @kesler_ksu  #рекомендации я кричу о том, чтобы каждый из вас смог понять, что счастье просто так не возьмешь. его нужно принять. просто так не пойдешь в магазин и не купишь его, как бы сильно не хотелось это взять. счастье — это что-то большое. это что-то родное, к чему нужно стремиться. не каждый раз, а опять и опять. и я хочу до тебя докричаться. и я хочу, чтобы ты смог понять, что близкие рядом — это те люди, которых стоит прямо сейчас обнять.
Предсказанные категории: Личная жизнь, Соцсети
Вероятности (по порядку категорий): Спорт: 0.01, Личная жизнь: 0.99, Юмор: 0.01, Соцсети: 0.88, Политика: 0.01, Реклама: 0.01, Нет категории: 0.01

Пример 2. Текст: потрясающее сырное печенье, очень рекомендую к приготовлению:  https://youtu.be/cw3zqugkd3q?si=rnasg5onpqh-y30y идеальное сырное печенье с сырным соусом по книге le cordon bleu 1904 года было приготовлено а посмотрите